In [1]:
import sys; sys.path.append("../../automl/")

In [2]:
from pathlib import Path
import yaml

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.lama import TabularLamaUtilized
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-05 17:42:47]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

## Constants

In [3]:
RANDOM_SEED = 77
DATA_PATH = Path("../../data/")
CONFIG_PATH = Path("../../configs/config.yaml")

In [4]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [5]:
create_ml_data_dir()
configure_root_logger()

## Data

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc.parquet")
df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [7]:
ohe_cols = df_train.columns[df_train.columns.str.startswith("OneHotEncoder")].values.tolist()
oe_cols = df_train.columns[df_train.columns.str.startswith("OrdinalEncoder")].values.tolist()
te_cols = df_train.columns[df_train.columns.str.startswith("MeanTargetEncoder")].values.tolist()

In [8]:
# take ordinal encoded columns
X_train, y_train = df_train[cfg["selected_features"] + oe_cols + ohe_cols + te_cols], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + oe_cols + ohe_cols + te_cols], df_test["target"]

In [9]:
display(y_train.value_counts(normalize=True))
display(y_test.value_counts(normalize=True))

0    0.940853
1    0.059147
Name: target, dtype: float64

0    0.941322
1    0.058678
Name: target, dtype: float64

In [10]:
categorical_features = ohe_cols + oe_cols

## Model

In [11]:
model = TabularLamaUtilized(timeout=60 * 60, task="classification", n_jobs=16, metric=RocAuc())

In [12]:
model.fit(X_train, y_train, categorical_features=categorical_features)

[2024-11-05 17:43:46,352] - [   START    ] - Fitting TabularLamaUtilized
[17:43:46] Start automl utilizator with listed constraints:
[17:43:46] - time: 7200.00 seconds
[17:43:46] - CPU: 16 cores
[17:43:46] - memory: 16 GB

[17:43:46] If one preset completes earlier, next preset configuration will be started

[17:43:46] ==================================================
[17:43:46] Start 0 automl preset configuration:
[17:43:46] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[17:43:46] Stdout logging level is INFO.
[17:43:46] Task: binary

[17:43:46] Start automl preset with listed constraints:
[17:43:46] - time: 7200.00 seconds
[17:43:46] - CPU: 16 cores
[17:43:46] - memory: 16 GB

[17:43:46] Train data shape: (300000, 77)

[17:43:58] Layer 1 train process start. Time left 7188.11 secs
[17:44:07] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[17:44:53] Fitting Lvl_0_Pipe

array([[0.9712412 , 0.02875882],
       [0.9814385 , 0.01856149],
       [0.89927167, 0.10072834],
       ...,
       [0.79000854, 0.20999143],
       [0.96663874, 0.03336123],
       [0.9555454 , 0.0444546 ]], dtype=float32)

## Inference best model

In [13]:
test = pd.read_parquet(DATA_PATH / "test_preproc.parquet")

In [14]:
test["target"] = model.predict(test[cfg["selected_features"] + oe_cols + ohe_cols + te_cols])[:, 1]

In [15]:
test[['id', 'target']].to_csv('lama_utilized.csv', index=False)

In [16]:
test["target"].describe()

count    171919.000000
mean          0.059728
std           0.067016
min           0.002106
25%           0.014638
50%           0.030828
75%           0.073483
max           0.531220
Name: target, dtype: float64

In [19]:
# blend

pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")

In [24]:
pred_1["target"] = 0.8 * pred_1["target"] + 0.2 * pred_2["target"]

In [26]:
pred_1.to_csv("blend.csv", index=False)

In [28]:
# stack 
#from sklearn.linear_model import LinearRegression
